<h1 style="text-align:center;"><strong>ASSIGNMENT 3</strong></h1>


<h4 style="text-align:center;"><strong>MOD300: Mandatory project 3</strong></h4>

<h4 style="text-align:center;"><strong>Group Members: Emad Omar Mohamed, Hassan Nehad Adnan, Sture Odin Domingos Troli</strong></h4>

<h5 style="text-align:center;">Nov 9, 2025</h5>

<h4 style="text-align:center;"><strong>Abstract</strong></h4>

In this project, we used Monte Carlo simulations to find the volume of a DNA molecule. We first tried the method on simple spheres, then added steps so it could also work for more complex molecules. For a sphere, we put random points in a 3D box and check if each point is inside or outside the sphere. The share of points inside tells us the sphere’s volume. We checked that the method works by working out π and comparing it to the known value.

Next, we used the same method on DNA. We treated each atom as a sphere with a known position and a van der Waals radius from the periodic table. Our results show that Monte Carlo simulations can find volumes well when we use enough random points. We also tried a random-walk method to measure the open space around the DNA. These results show that Monte Carlo methods are useful for tough shape problems in molecular biology. We also talk about how close our results are to the real answer, where errors might come from, and how we could make the method better in the future.

#### **TOPIC 1: Calculate DNA volume via Monte Carlo simulation.**

Task 0 - 10

this is a test emad123


#### **TOPIC 2: Random walk for accessible volume calculation**

Task 1 - 5

#### **8 Conclusion and Discussion**

## Final reflections


#### **9 Self-reflections**
**Self-reflections.**

**Emad Omar Mohamed**

**Hassan Nehad Adnan**

**Sture Odin Domingos Troli**

**Collective reflection:**

#### **References**
